# `fuzzymatcher` examples

## Basic usage - `link_table`

In the most basic usage, the user provides `fuzzymatcher` with two pandas dataframes, indicating which columns to join on.

The central output of `fuzzymatcher` is the `link_table`.

For each record in the left table, the link table includes one or more possible matching records from the right table.

The user can then inspect the link table and decide which matches to retain, e.g. by choosing a score threshold ( `match_score > chosen_threshold` ) or just choosing the best match ( `match_rank == 1` )

In [ ]:
import logging
logging.basicConfig(level=logging.DEBUG)

In [ ]:
import fuzzymatcher
import pandas as pd

df_left = pd.read_csv("tests/data/left_1.csv")
df_left

In [ ]:
df_right = pd.read_csv("tests/data/right_1.csv")
df_right

In [ ]:
# Columns to match on from df_left
left_on = ["fname", "mname", "lname",  "dob"]

# Columns to match on from df_right
right_on = ["name", "middlename", "surname", "date"]

# Note that if left_id_col or right_id_col are admitted a unique id will be autogenerated
fuzzymatcher.link_table(df_left, df_right, left_on, right_on, left_id_col = "id", right_id_col = "id")

## Basic usage - `fuzzy_left_join`

A second option is to use `fuzzy_left_join`, which automatically links the two dataframes based on the highest-scoring match.

In [5]:
import fuzzymatcher
import pandas as pd

df_left = pd.read_csv("tests/data/left_1.csv")
df_right = pd.read_csv("tests/data/right_1.csv")
left_on = ["fname", "lname",  "dob"]
right_on = ["name", "surname", "date"]

fuzzymatcher.fuzzy_left_join(df_left, df_right, left_on, right_on)

DEBUG:fuzzymatcher.matcher:Matching 4 left records against 3 right records
DEBUG:fuzzymatcher.matcher:Processed 4 records, 100% done


,best_match_score,__id_left,__id_right,id_left,fname,mname,lname,dob,another_field,id_right,name,middlename,surname,date,other
0,0.101885,0_left,0_right,1,Will,James,Johnston,20/05/1980,other data,1.0,William,J,Johnston,20/05/1980,other data
2,0.225707,1_left,1_right,2,James,Paul,Smith,15/06/1990,more data,2.0,James,Paul,Smith,15/06/1990,more data
3,0.066847,2_left,2_right,3,Jody,Liz,Brown,20/05/1960,another thing,3.0,Jodi,Elizabeth,Brown,20/05/1961,another thing
5,NaN,3_left,None,4,David,James,Williams,01/01/2000,final thing,NaN,NaN,NaN,NaN,NaN,NaN


# Basic example with real data
### Matching the names of Local Authorities provided by Office for National Statistics with the names provided by Ordnance Survey

We would usually join this data on the Local Authority District (LAD) Codes (e.g. E06000001 = Hartlepool), but sometimes these are unavailable.  In this example, we fuzzy match on the name, but provide the LAD code to demonstate it has worked.

In [6]:
ons = pd.read_csv("tests/data/las_ons.csv")
os = pd.read_csv("tests/data/las_os.csv")

df_joined = fuzzymatcher.fuzzy_left_join(ons, os, left_on = "lad16nm", right_on = "name")
rename = {"lad16cd": "ons_code", "code": "os_code", "lad16nm": "ons_name", "name": "os_name"}
df_joined = df_joined.rename(columns=rename)
col_order = ["best_match_score", "ons_name", "os_name", "ons_code", "os_code"]
df_joined[col_order].head()

DEBUG:fuzzymatcher.matcher:Matching 380 left records against 380 right records
DEBUG:fuzzymatcher.matcher:Processed 249 records, 66% done
DEBUG:fuzzymatcher.matcher:Processed 380 records, 100% done


,best_match_score,ons_name,os_name,ons_code,os_code
0,0.178449,Hartlepool,Hartlepool (B),E06000001,E06000001
1,0.178449,Middlesbrough,Middlesbrough (B),E06000002,E06000002
2,0.498368,Redcar and Cleveland,Redcar and Cleveland (B),E06000003,E06000003
3,0.543143,Stockton-on-Tees,Stockton-on-Tees (B),E06000004,E06000004
8,0.178449,Darlington,Darlington (B),E06000005,E06000005


We can get a sense of match quality by measuring how often the fuzzy matcher got it right:

In [7]:
num_records = len(df_joined)
correct_binary = (df_joined["ons_code"] == df_joined["os_code"])
perc_correct = correct_binary.sum()/num_records

"The percentage of codes which were correctly matched was {:,.1f}%".format(perc_correct*100)

'The percentage of codes which were correctly matched was 99.7%'

# Advanced usage - configuring the matcher

`fuzzymatcher` uses a number of components, each one of which can be re-written or adapted by the user:

* **`data_preprocessor`**:  Responsible for normalising strings, removing punctuation etc.
* **`datagetter`**:  Responsible for finding a list of possible matches for each df_left record in df_right
* **`scorer`**:  Responsible for computing a match score, given a record from df_left and df_right respectively

The main `link_table` and `fuzzy_left_join` convenience functions use these components under the hood.  See [here](https://github.com/RobinL/fuzzymatcher/blob/master/fuzzymatcher/__init__.py) for how this work.

This section provides a few examples of how an advanced user can compose these components to create a custom matcher

## Example 1:  Replacing the default sqlite datagetter with the cartesian datagetter

In [8]:
from fuzzymatcher.data_getter_cartesian import DataGetterCartesian
from fuzzymatcher.matcher import Matcher

dg = DataGetterCartesian()

m = Matcher(data_getter = dg)

df_left = pd.read_csv("tests/data/left_3.csv")
df_right = pd.read_csv("tests/data/right_3.csv")

on = ["first_name", "surname", "dob", "city"]

m.add_data(df_left, df_right, on, on)

m.match_all()
lt = m.get_formatted_link_table()
print("Length of Cartesian join table: {}".format(len(lt))) # Note, because df_left and df_right are 100 records each, this table is 10,000 records long
lt.head()

DEBUG:fuzzymatcher.matcher:Matching 100 left records against 100 right records
DEBUG:fuzzymatcher.matcher:Processed 100 records, 100% done


Length of Cartesian join table: 100


,__id_left,__id_right,match_score,match_rank,first_name_left,first_name_right,surname_left,surname_right,dob_left,dob_right,city_left,city_right
0,0_left,None,None,1,Noah,NaN,John,NaN,1979-05-29,NaN,London,NaN
1,1_left,None,None,1,William,NaN,Leo,NaN,1996-01-29,NaN,Ipswich,NaN
2,2_left,None,None,1,Muhammad,NaN,lRey,NaN,1987-04-13,NaN,Bradford,NaN
3,3_left,None,None,1,iasJne,NaN,Newton,NaN,1973-09-15,NaN,London,NaN
4,4_left,None,None,1,Brown,NaN,Jacob,NaN,1994-10-11,NaN,London,NaN


The Cartesian matcher considers more potential matches, but its performance is considerably worse

# Performance


In [ ]:

df_left = pd.read_csv("tests/data/left_4.csv")
df_right = pd.read_csv("tests/data/right_4.csv")

on = ["first_name", "surname", "dob", "city"]

flj = fuzzymatcher.fuzzy_left_join(df_left, df_right, on, on)
flj.head()

DEBUG:fuzzymatcher.matcher:Matching 10000 left records against 10000 right records
DEBUG:fuzzymatcher.matcher:Processed 249 records, 2% done
DEBUG:fuzzymatcher.matcher:Processed 499 records, 5% done
DEBUG:fuzzymatcher.matcher:Processed 749 records, 7% done
DEBUG:fuzzymatcher.matcher:Processed 999 records, 10% done
DEBUG:fuzzymatcher.matcher:Processed 1249 records, 12% done
DEBUG:fuzzymatcher.matcher:Processed 1499 records, 15% done
DEBUG:fuzzymatcher.matcher:Processed 1749 records, 17% done
DEBUG:fuzzymatcher.matcher:Processed 1999 records, 20% done
DEBUG:fuzzymatcher.matcher:Processed 2249 records, 22% done
DEBUG:fuzzymatcher.matcher:Processed 2499 records, 25% done
DEBUG:fuzzymatcher.matcher:Processed 2749 records, 27% done
DEBUG:fuzzymatcher.matcher:Processed 2999 records, 30% done
DEBUG:fuzzymatcher.matcher:Processed 3249 records, 32% done
DEBUG:fuzzymatcher.matcher:Processed 3499 records, 35% done
DEBUG:fuzzymatcher.matcher:Processed 3749 records, 37% done
DEBUG:fuzzymatcher.match